# **CONNECT WITH DRIVE**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**IMPORT DEPENDENCIES**

In [2]:


!pip install -q numpy pandas opencv-python matplotlib seaborn tensorflow keras pillow plotly && apt-get update && apt-get install -y libgl1-mesa-glx;

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,242 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,498 kB]
Get:13 https://ppa.launchpadcontent.net/c2d4u.tea

**IMPORT LIBRARIES**

In [7]:
import numpy as np                 # Importing the NumPy library for numerical computations
import pandas as pd                # Importing the Pandas library for data manipulation and analysis
import os                          # Importing the os module for interacting with the operating system

import cv2                         # Importing the OpenCV library for image processing

import matplotlib.pyplot as plt    # Importing the Matplotlib library for data visualization
import seaborn as sns

import tensorflow as tf            # Importing the TensorFlow library for deep learning
from tensorflow import keras       # Importing the Keras module from TensorFlow for building neural networks
from PIL import Image              # Importing the PIL library for image processing
from tensorflow.keras.optimizers import Adam

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from keras.utils import to_categorical

from sklearn.model_selection import train_test_split  # Importing train_test_split from scikit-learn for splitting data
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Importing ImageDataGenerator for data augmentation
from tensorflow.keras.optimizers import Adam   # Importing the Adam optimizer from TensorFlow for model optimization

from sklearn.metrics import accuracy_score    # Importing accuracy_score from scikit-learn for calculating accuracy
from matplotlib import style       # Importing style from Matplotlib for customizing plot styles

import warnings
warnings.filterwarnings("ignore")  # Ignore warning messages during code execution
np.random.seed(42)                # Setting the random seed for reproducibility

**DATA PREPARATION**

In [8]:
#FUNCTION TO READ IMAGES FROM DIRECTORY
def read_images(directory_path):
    images = []
    labels = []

    valid_extensions = ('.jpg', '.jpeg', '.png')  # Valid image file extensions

    # Get subdirectories within the directory
    subdirectories = [subdir for subdir in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, subdir))]

    for subdir in subdirectories:
        subdirectory_path = os.path.join(directory_path, subdir)
        if not os.listdir(subdirectory_path):
            continue  # Skip empty subdirectories

        # Iterate over images in the subdirectory
        for filename in os.listdir(subdirectory_path):
            if filename.lower().endswith(valid_extensions):  # Check if the file has a valid image extension
                image_path = os.path.join(subdirectory_path, filename)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (30, 30))  # Resize the image to 30x30 pixels
                images.append(image)

                # Assign the label based on the subdirectory name
                label = int(subdir)
                labels.append(label)

    data = np.array(list(zip(images, labels)))

    return data

In [9]:
train_data = read_images('/content/drive/MyDrive/CAPSTONE2/GTSR/Train')

In [10]:
print(f"Train Data Shape : {train_data.shape}")

Train Data Shape : (29433, 2)


In [11]:
test_data = read_images('/content/drive/MyDrive/CAPSTONE2/GTSR/Test')

In [14]:
print(f"Test Data Shape : {test_data.shape}")

Test Data Shape : (0,)


In [13]:
meta_data = read_images('/content/drive/MyDrive/CAPSTONE2/GTSR/Meta')

In [15]:
print(f"meta Data Shape : {meta_data.shape}")

meta Data Shape : (0,)


 **CLASS LABEL**

In [16]:
# Label Overview
classes = {
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)',
    2:'Speed limit (50km/h)',
    3:'Speed limit (60km/h)',
    4:'Speed limit (70km/h)',
    5:'Speed limit (80km/h)',
    6:'End of speed limit (80km/h)',
    7:'Speed limit (100km/h)',
    8:'Speed limit (120km/h)',
    9:'No passing',
    10:'No passing veh over 3.5 tons',
    11:'Right-of-way at intersection',
    12:'Priority road',
    13:'Yield',
    14:'Stop',
    15:'No vehicles',
    16:'Veh > 3.5 tons prohibited',
    17:'No entry',
    18:'General caution',
    19:'Dangerous curve left',
    20:'Dangerous curve right',
    21:'Double curve',
    22:'Bumpy road',
    23:'Slippery road',
    24:'Road narrows on the right',
    25:'Road work',
    26:'Traffic signals',
    27:'Pedestrians',
    28:'Children crossing',
    29:'Bicycles crossing',
    30:'Beware of ice/snow',
    31:'Wild animals crossing',
    32:'End speed + passing limits',
    33:'Turn right ahead',
    34:'Turn left ahead',
    35:'Ahead only',
    36:'Go straight or right',
    37:'Go straight or left',
    38:'Keep right',
    39:'Keep left',
    40:'Roundabout mandatory',
    41:'End of no passing',
    42:'End no passing veh > 3.5 tons'
}

**Images with their corresponding class.**

In [17]:
labels = train_data[:, 1].astype(int)  # Extract the labels from the train_data array
unique_labels, label_counts = np.unique(labels, return_counts=True)
count_dict= {label: count for label, count in zip(unique_labels, label_counts)}
print(count_dict)

{0: 210, 1: 1140, 2: 1120, 3: 1110, 4: 1100, 5: 1100, 6: 420, 7: 1090, 8: 1090, 9: 1090, 10: 1140, 11: 1134, 12: 1130, 13: 1130, 14: 780, 15: 630, 16: 420, 17: 1110, 18: 1120, 19: 210, 20: 360, 21: 330, 22: 390, 23: 510, 24: 270, 25: 1120, 26: 600, 27: 240, 28: 540, 29: 270, 30: 450, 31: 780, 32: 240, 33: 689, 34: 420, 35: 1110, 36: 390, 37: 210, 38: 1100, 39: 300, 40: 360, 41: 240, 42: 240}


**EDA** - To analysis the data


In [42]:
# Define the base color
base_color = '#FF69B4'  # Red-pink color

# Create a color scale using the base color
color_scale = [[0, base_color], [1, 'rgb(30,30,30)']]  # Start with the base color and end with dark background

# Create a horizontal bar plot
fig = go.Figure(data=[go.Bar(
    x=label_counts,
    y=[classes[label] for label in unique_labels],
    orientation='h',
    marker=dict(
        color=label_counts,
        coloraxis='coloraxis',

    )
)])

# Customize the layout
fig.update_layout(
    title='Label Counts in Train Data',
    xaxis_title='Count',
    yaxis_title='Label',

    plot_bgcolor='rgb(30,30,30)',  # Dark background color
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    font=dict(color='white', size=6),  # White font color
    coloraxis=dict(colorscale=color_scale)  # Color scale for bars
)

# Show the plot
fig.show()


In [20]:
# Define the base color and generate a color palette
base_color = '#8A2BE2'
color_palette = sns.color_palette("husl", len(count_dict)).as_hex()

# Create the pie plot
fig = go.Figure(data=[go.Pie(
    labels=list(count_dict.keys()),
    values=list(count_dict.values()),
    marker=dict(colors=color_palette),
)])

# Update the layout
fig.update_layout(
    title='Label Distribution',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()

**Inference**
18 class labels have more iamges around 1100plus e.g Keep rught , Speed limit 120,100,70,60,50,30 , stop etc

```



In [43]:
# Select 10 random samples from train_data
indices = np.random.choice(train_data.shape[0], size=20, replace=False)
samples = train_data[indices]

# Create a grid of subplots for the images
fig = make_subplots(rows=4, cols=5, subplot_titles=[classes[label] for _, label in samples])

for i, (image, label) in enumerate(samples):
    row = i // 5 + 1
    col = i % 5 + 1

    # Add the image to the subplot
    fig.add_trace(
        go.Image(z=image),
        row=row,
        col=col
    )

# Update the layout
fig.update_layout(
    title='Random Samples',
    font=dict(color='white'),
    paper_bgcolor='rgb(40,40,40)',  # Dark background color
    plot_bgcolor='rgb(40,40,40)'  # Dark background color
)

# Show the plot
fig.show()

In [24]:
train_ = pd.DataFrame(train_data, columns=['Image', 'Label'])
train_.head()

,Image,Label
0,"[[[67, 67, 68], [73, 72, 77], [84, 84, 95], [1...",0
1,"[[[63, 63, 67], [65, 66, 69], [68, 72, 82], [8...",0
2,"[[[98, 95, 114], [137, 129, 200], [71, 74, 183...",0
3,"[[[75, 73, 73], [75, 73, 74], [69, 72, 75], [8...",0
4,"[[[67, 69, 73], [88, 92, 130], [109, 111, 184]...",0


## FIXING NUMBER OF SAMPLES IMAGES IN EACH CLASS

In [49]:
class_count = 200  # Number of samples to select from each class as lowest in 210
grouped = train_.groupby('Label')# Group the DataFrame by the 'y' column

# Use the apply function to select the desired number of samples from each class
balanced_df = grouped.apply(lambda x: x.sample(class_count))

# Reset the index of the balanced DataFrame
balanced_df.reset_index(drop=True, inplace=True)

In [45]:
balanced_df.shape ## REDUCE FROM 29888 to 8600

(8600, 2)

In [46]:
train_balanced = balanced_df.values

In [47]:
new_labels = train_balanced[:, 1].astype(int)  # Extract the labels from the train_data array
new_unique_labels, new_label_counts = np.unique(new_labels, return_counts=True)
new_count_dict= {label: count for label, count in zip(new_unique_labels, new_label_counts)}
print(new_count_dict)

{0: 200, 1: 200, 2: 200, 3: 200, 4: 200, 5: 200, 6: 200, 7: 200, 8: 200, 9: 200, 10: 200, 11: 200, 12: 200, 13: 200, 14: 200, 15: 200, 16: 200, 17: 200, 18: 200, 19: 200, 20: 200, 21: 200, 22: 200, 23: 200, 24: 200, 25: 200, 26: 200, 27: 200, 28: 200, 29: 200, 30: 200, 31: 200, 32: 200, 33: 200, 34: 200, 35: 200, 36: 200, 37: 200, 38: 200, 39: 200, 40: 200, 41: 200, 42: 200}


## **EDA OF NEW BALANCED DATA**

In [50]:
# Define the base color and generate a color palette
base_color = '#8A2BE2'
color_palette = sns.color_palette("husl", len(new_count_dict)).as_hex()

# Create the pie plot
fig = go.Figure(data=[go.Pie(
    labels=list(new_count_dict.keys()),
    values=list(new_count_dict.values()),
    marker=dict(colors=color_palette),
)])

# Update the layout
fig.update_layout(
    title='Label Distribution',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()

In [51]:
shuffled_train = train_balanced.copy()     # Create a copy of train_data to avoid modifying the original array
shuffled_train = np.array(shuffled_train)  # Shuffle the array randomly

In [52]:
# Split the shuffled_train array into training and testing sets
train_set, test_set = train_test_split(shuffled_train, test_size=0.2, random_state=42)

# Separate the input (X) and output/label (y) arrays from the training set
x_train = train_set[:, 0]
y_train = train_set[:, 1]

# Separate the input (X) and output/label (y) arrays from the testing set
x_test = test_set[:, 0]
y_test = test_set[:, 1]

# Convert the data type of the arrays to int
x_train = np.array([np.array(x) for x in x_train])
y_train = np.array([np.array(y) for y in y_train])
x_test = np.array([np.array(x) for x in x_test])
y_test = np.array([np.array(y) for y in y_test])

# Normalize the input data
x_train = x_train / 255.0
x_test  = x_test / 255.0

print("x_train shape:", x_train.shape)
print("x_test  shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test  shape:", y_test.shape)

x_train shape: (6880, 30, 30, 3)
x_test  shape: (1720, 30, 30, 3)
y_train shape: (6880,)
y_test  shape: (1720,)


In [53]:
# Convert target labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=43)
y_test  = to_categorical(y_test, num_classes=43)

## Model Creation and Evaluation

In [54]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(30,30,3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),

    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(43, activation='softmax')
])

In [55]:
lr = 0.001
epochs = 30
opt = Adam(learning_rate=lr)  # Set the learning_rate instead of decay
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [56]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=15,            # Increase the rotation range for more varied rotations
    zoom_range=0.2,               # Increase the zoom range for more varied zoom levels
    width_shift_range=0.15,       # Increase the range of horizontal shift
    height_shift_range=0.15,      # Increase the range of vertical shift
    shear_range=0.2,              # Increase the shear range for more shearing effects
    horizontal_flip=True,         # Enable horizontal flipping
    vertical_flip=True,           # Enable vertical flipping
    fill_mode="reflect"           # Use reflect mode for filling empty pixels
)


history = model.fit(aug.flow(x_train,y_train, batch_size=32), epochs=epochs, validation_data=(x_test,y_test))

Epoch 1/30
215/215 [==============================] - 8s 24ms/step - loss: 3.5628 - accuracy: 0.1584 - val_loss: 4.0510 - val_accuracy: 0.0326
Epoch 2/30
215/215 [==============================] - 5s 23ms/step - loss: 2.4080 - accuracy: 0.3251 - val_loss: 3.7920 - val_accuracy: 0.1169
Epoch 3/30
215/215 [==============================] - 5s 22ms/step - loss: 1.9157 - accuracy: 0.4206 - val_loss: 1.6808 - val_accuracy: 0.4605
Epoch 4/30
215/215 [==============================] - 8s 36ms/step - loss: 1.5218 - accuracy: 0.5113 - val_loss: 0.9539 - val_accuracy: 0.6535
Epoch 5/30
215/215 [==============================] - 4s 21ms/step - loss: 1.2731 - accuracy: 0.5770 - val_loss: 1.0220 - val_accuracy: 0.6547
Epoch 6/30
215/215 [==============================] - 6s 29ms/step - loss: 1.1058 - accuracy: 0.6308 - val_loss: 0.7186 - val_accuracy: 0.7564
Epoch 7/30
215/215 [==============================] - 5s 23ms/step - loss: 0.9482 - accuracy: 0.6789 - val_loss: 0.5685 - val_accuracy: 0.8151

In [57]:
# Create the figure
fig = go.Figure()

# Add the training loss trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['loss'],
    mode='lines',
    name='Training Loss'
))

# Add the validation loss trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['val_loss'],
    mode='lines',
    name='Validation Loss'
))

# Update the layout
fig.update_layout(
    title='Model Loss',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()

In [58]:
# Create the figure
fig = go.Figure()

# Add the training accuracy trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['accuracy'],
    mode='lines',
    name='Training Accuracy'
))

# Add the validation accuracy trace
fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['val_accuracy'],
    mode='lines',
    name='Validation Accuracy'
))

# Update the layout
fig.update_layout(
    title='Model Accuracy',
    xaxis_title='Epochs',
    yaxis_title='Accuracy',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',  # Dark background color
    plot_bgcolor='rgb(30,30,30)'  # Dark background color
)

# Show the plot
fig.show()

In [59]:
# Convert NumPy array to DataFrame
shuffled_train_dataframe = pd.DataFrame(shuffled_train)
# Save DataFrame as CSV file
shuffled_train_dataframe.to_csv('data.csv', index=False)

In [60]:
# Saving the Model
model.save('traffic_classifier.h5')

In [61]:
def predict_image(model_path, image_path):
    # Load the model
    model = tf.keras.models.load_model(model_path)

    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (30, 30))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0

    # Make predictions
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)

    return predicted_label

model_path = "traffic_classifier.h5"
image_path = "/content/drive/MyDrive/CAPSTONE2/GERMAN TRAFFIC SIGN/Test/00251.png"

predicted_label = predict_image(model_path, image_path)
print("Predicted Label:", classes[predicted_label])

1/1 [==============================] - 0s 234ms/step
Predicted Label: Stop


### **Conclusion**

The traffic sign recognition model notebook provides a comprehensive analysis and prediction model for recognizing traffic signs. By using deep learning techniques and the provided dataset, the notebook demonstrates how to preprocess the images, build a convolutional neural network (CNN) model, and evaluate its performance using accuracy as the metric. The notebook also showcases the effectiveness of data augmentation techniques in improving the model's performance.

The CNN model architecture consists of several convolutional and pooling layers, followed by fully connected layers and a softmax activation function for multi-class classification. The model is trained using the Adam optimizer and categorical cross-entropy loss function.

By training the model on the traffic sign dataset, the notebook demonstrates how to achieve high accuracy in recognizing and classifying different traffic signs. The model can be used in real-world applications to aid in automated traffic sign detection and recognition systems.

Overall, this notebook serves as a valuable resource for understanding and implementing traffic sign recognition using deep learning techniques.